In [57]:
import os
import json
import pandas as pd
from chardet import detect


def read_json_file(path):
    try: 
        with open(path, 'r') as file:
            return json.load(file)
    except UnicodeDecodeError: 
        return
        

def create_dataframe_from_frames(frames, joint_names):
    all_data = []
    for frame in frames:
        for body in frame['bodies']:
            row_data = {'frame_id': frame['frame_id'], 'timestamp_unix': frame['timestamp_unix']}
            for i, joint_name in enumerate(joint_names):
                row_data[f'{joint_name}_x'] = body['joint_positions'][i][0]
                row_data[f'{joint_name}_y'] = body['joint_positions'][i][1]
                row_data[f'{joint_name}_z'] = body['joint_positions'][i][2]
                row_data[f'{joint_name}_c'] = body['joint_confidence'][i]
            all_data.append(row_data)
    
    df = pd.DataFrame(all_data)
    df['timestamp'] = pd.to_datetime(df['timestamp_unix'], unit='us')
    df.set_index('timestamp', inplace=True)
    return df

def interpolate_data(df):
    numeric_columns = df.select_dtypes(include=['float64']).columns
    df_interpolated = df[numeric_columns].interpolate(method='linear')
    df_interpolated['frame_id'] = df['frame_id'].astype(int)
    return df_interpolated

def process_and_save_csv(json_path, output_path):
    json_data = read_json_file(json_path)
    if json_data is None:
        return  
    frames = json_data['frames']
    joint_names = json_data['joint_names']
    
    df = create_dataframe_from_frames(frames, joint_names)
    df_interpolated = interpolate_data(df)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    df_interpolated.to_csv(output_path)

def process_folder(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        json_path = os.path.join(input_folder, filename)
        csv_filename = filename + '.csv'
        output_path = os.path.join(output_folder, csv_filename)
        process_and_save_csv(json_path, output_path)

# Example usage
input_folder = '/Users/fjonalutaj/Desktop/data'
output_folder = '/Users/fjonalutaj/Desktop/processed_data'
process_folder(input_folder, output_folder)
